In [1]:
#importing the Training data
import pandas as pd
data=pd.read_csv('train.csv')
data.head()

,ID,DateTime,TempOut,HiTemp,LowTemp,OutHum,DewPt,WindSpeed,WindDir,WindRun,...,WindTx,ISSRecpt,ArcInt,PA,PB,PC,PD,PE,PF,PG
0,PR00001,07-12-40 0:15,53.5,53.6,53.5,85,49.1,2,SSE,0.5,...,1,100.0,15,1,1,1,1,1,1,1
1,PR00002,07-12-40 0:30,53.5,53.5,53.4,85,49.1,2,SSE,0.5,...,1,100.0,15,1,1,1,1,1,1,1
2,PR00003,07-12-40 0:45,53.3,53.5,53.2,85,48.9,2,SSE,0.5,...,1,100.0,15,1,1,1,1,1,1,1
3,PR00004,07-12-40 1:00,53.1,53.3,53.0,86,49.0,2,S,0.5,...,1,100.0,15,1,1,1,1,1,1,1
4,PR00005,07-12-40 1:15,52.9,53.1,52.9,86,48.8,2,S,0.5,...,1,100.0,15,1,1,1,1,1,1,1


In [2]:
#Checking the correlation of data
corr_mat=data.corr()
corr_mat.head()

,TempOut,HiTemp,LowTemp,OutHum,DewPt,WindSpeed,WindRun,HiSpeed,WindChill,HeatIndex,...,WindTx,ISSRecpt,ArcInt,PA,PB,PC,PD,PE,PF,PG
TempOut,1.000000,0.999016,0.997934,-0.760999,0.514769,0.505322,0.505322,0.543677,0.999008,0.997370,...,NaN,-0.002226,NaN,0.191000,0.187726,0.183378,0.178264,0.172692,0.167018,0.160992
HiTemp,0.999016,1.000000,0.997799,-0.766949,0.508606,0.511188,0.511188,0.549135,0.997832,0.995947,...,NaN,-0.001991,NaN,0.189183,0.185948,0.181650,0.176595,0.171085,0.165473,0.159520
LowTemp,0.997934,0.997799,1.000000,-0.753150,0.519205,0.511173,0.511173,0.547212,0.996875,0.995686,...,NaN,-0.002441,NaN,0.192722,0.189445,0.185079,0.179935,0.174330,0.168609,0.162535
OutHum,-0.760999,-0.766949,-0.753150,1.000000,0.123583,-0.483666,-0.483666,-0.513849,-0.753988,-0.719567,...,NaN,0.000475,NaN,-0.030012,-0.028772,-0.027217,-0.025561,-0.023830,-0.021923,-0.020060
DewPt,0.514769,0.508606,0.519205,0.123583,1.000000,0.178021,0.178021,0.193896,0.520461,0.567098,...,NaN,-0.004275,NaN,0.273815,0.272070,0.268943,0.264630,0.259584,0.254393,0.248483


In [ ]:
#Creating the Modified train data or feature engineering
from sklearn.preprocessing import LabelEncoder
from dateutil import parser
le=LabelEncoder()
data['WindDir']=le.fit_transform(data['WindDir'])
data['HiDir']=le.fit_transform(data['HiDir'])
a = lambda x:pd.to_numeric(parser.parse(x).timestamp())
data.DateTime=data.DateTime.apply(a)
data_sample=data.copy()
data_sample.head()
#droping 'ArcInt' and 'WindTx' since they have no correlation
data_sample=data_sample.drop(['ArcInt','WindTx'],axis=1)
data_sample.to_csv('Modified_train.csv')
data_sample.head()

In [6]:
data_sample=pd.read_csv('Modified_train.csv')
data_sample.drop(['Unnamed: 0','ID'],axis=1,inplace=True)
print (data_sample.columns)
X=data_sample.iloc[:,0:27]
Y=data_sample.iloc[:,27:]
print (X.columns)  #seperating Train/Test features
print (Y.columns)

Index(['DateTime', 'TempOut', 'HiTemp', 'LowTemp', 'OutHum', 'DewPt',
       'WindSpeed', 'WindDir', 'WindRun', 'HiSpeed', 'HiDir', 'WindChill',
       'HeatIndex', 'THWIndex', 'Bar', 'Rain', 'RainRate', 'HeatDD', 'CoolDD',
       'InTemp', 'InHum', 'InDew', 'InHeat', 'InEMC', 'InAirDensity',
       'WindSamp', 'ISSRecpt', 'PA', 'PB', 'PC', 'PD', 'PE', 'PF', 'PG'],
      dtype='object')
Index(['DateTime', 'TempOut', 'HiTemp', 'LowTemp', 'OutHum', 'DewPt',
       'WindSpeed', 'WindDir', 'WindRun', 'HiSpeed', 'HiDir', 'WindChill',
       'HeatIndex', 'THWIndex', 'Bar', 'Rain', 'RainRate', 'HeatDD', 'CoolDD',
       'InTemp', 'InHum', 'InDew', 'InHeat', 'InEMC', 'InAirDensity',
       'WindSamp', 'ISSRecpt'],
      dtype='object')
Index(['PA', 'PB', 'PC', 'PD', 'PE', 'PF', 'PG'], dtype='object')


In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3)
print(x_train.shape)
print(y_train.shape)

(28000, 27)
(28000, 7)


In [8]:
#Evaluation using Randomforest
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(n_estimators=400,max_depth=40)
from sklearn.multioutput import MultiOutputRegressor
mul_rf = MultiOutputRegressor(rf_reg)
mul_rf.fit(x_train, y_train)
y_pred = mul_rf.predict(x_test)
#check r2 score for one column
from sklearn.metrics import r2_score
r2_score(y_test["PC"].tolist(), y_pred[:, 2].tolist())

0.9840153155840679

In [10]:
#Loading and cleaning the Test File
test_df=pd.read_csv('	est.csv')
print (test_df.shape)
from dateutil import parser
b=lambda x:pd.to_numeric(parser.parse(x).timestamp())
test_df['DateTime']=test_df['DateTime'].apply(b)
test_df['WindDir']=le.fit_transform(test_df['WindDir'])
test_df['HiDir']=le.fit_transform(test_df['HiDir'])
test_df=test_df.drop(['ArcInt','WindTx'],axis=1)
test_df_f = test_df.iloc[:, 1:]
print (test_df_f.shape)

(22446, 30)


NameError: name 'le' is not defined

In [ ]:
#Creating submission file
test_pred=mul_rf.predict(test_df_f)
test_pred_result=pd.DataFrame(test_pred)
print (test_pred_result.head())
test_pred_result.columns=['PA','PB','PC','PD','PE','PF','PG']
test_pred_result.insert(0,'ID',test_df_id)
test_pred_result = test_pred_result.round(0)
print(test_pred_result.head())
test_pred_result.to_csv('submission.csv',index=False)